In [1]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller

In [2]:
# Lecture du fichier *.txt
data = pd.read_csv('data.txt', sep=',').set_index(keys='Date')

In [3]:
# Niveau de confiance alpha de 5%
alpha = .05
# Application du test Augmented Dickey-Fuller aux variables du dataset
adf_results = {}
for col in data.columns:
    # Récupération de la p-valeur du test
    pval = adfuller(data[col])[1]
    # Indicatrice de stationnarité
    hadf = pval < alpha
    # Collecte des résultats
    adf_results[col] = [round(pval, 3) * 100, hadf]
# Affichage des résultats
pd.DataFrame(adf_results, index=['p-value (%)', 'I0'])

,CPI,INDPRO,RPI,UNRATE
p-value (%),51.8,28.6,99.7,1.2
I0,False,False,False,True


In [4]:
# Sélection des variables non stationnaires
ns_data = data.loc[:, data.columns != 'UNRATE']
# Calcul des séries en différence première
diff_data = ns_data.diff(periods=1)
# Aggrégation des variables stationnaires 
mapping = pd.concat([diff_data, data['UNRATE']], axis=1).dropna()
# Centrage et standardisation des données
std_data = (mapping - mapping.mean(axis=0)) / mapping.std(axis=0)

In [5]:
# Séléction de la période d'estimation
estimation_sample = std_data.loc[(std_data.index >= '1983-01-01') & (std_data.index <= '2016-12-01')]
# Sélection de la période de prévision
forecast_sample = std_data.loc[std_data.index >= '2017-01-01']